In [1]:
# for importing data
import pandas as pd
import numpy as np
import os

# for EDA
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
%matplotlib inline                                                                                      

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
# Use cleaner absolute path to find file
path = os.path.abspath('politifact_cleaned_data.csv')
df = pd.read_csv(path, index_col=0)

# to see statement in full
pd.set_option('display.max_colwidth', 200) 

# See sample of data
df.head(3)

In [ ]:
# Start with one document
text = docdf.document[0]

# Create and generate a word cloud image:
wordcloud = WordCloud().generate(text)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# Save the image in the img folder:
# Uncomment this to save file locally when it runs
#wordcloud.to_file("images/first_doc_allwords_cloud.png")

# Make it look better
# lower max_font_size, change the maximum number of word and lighten the background:
wordcloud = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(text)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# Save the image in the img folder:
# Uncomment this to save file locally when it runs
#wordcloud.to_file("images/first_doc_cloud_100words.png")

# Combine all documents into a single word cloud
text = " ".join(review for review in docdf.document) # Need to update words here
print ("There are {} words in the combination of all documents.".format(len(text)))

# Create stopword list:
# Skip this for now need to figure out which stopwords if needed
# stopwords = set(STOPWORDS)
# stopwords.update(["stopone", "stoptwo", "stopthree", "stopfour", "stopfive"])

# Generate a word cloud image
wordcloud = WordCloud(background_color="white").generate(text)
# When stopwords added change to this: wordcloud = WordCloud(stopwords=stopwords, background_color="white").generate(text)

# Display the generated image:
# the matplotlib way:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

# Save the image in the img folder:
# Uncomment this to save file locally when it runs
#wordcloud.to_file("images/all_doc_cloud.png")